In [2]:
!pip install transformers datasets wandb torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from datasets import load_dataset

# 📥 Lade den Poetry-Datensatz von Hugging Face (enthält englische Gedichte)
dataset = load_dataset("biglam/gutenberg-poetry-corpus", split="train")




README.md:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/903 [00:00<?, ?B/s]

(…)-00000-of-00001-fa9fb9e1f16eed7e.parquet:   0%|          | 0.00/91.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3085117 [00:00<?, ? examples/s]

KeyError: 'text'

In [8]:
# 🖨️ Zeige das erste Gedicht im Datensatz
print(dataset[0]["line"])

The Song of Hiawatha is based on the legends and stories of


In [9]:
# Verbinde alle Zeilen zu einem einzigen Text
all_poems = "\n".join(example["line"] for example in dataset)

# Speichere den Text in einer Datei
with open("gutenberg_poems.txt", "w", encoding="utf-8") as f:
    f.write(all_poems)


In [10]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 не имеет pad_token

model = GPT2LMHeadModel.from_pretrained("gpt2")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
def tokenize_function(example):
    return tokenizer(example["line"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3085117 [00:00<?, ? examples/s]

In [15]:
!pip install --upgrade transformers

In [21]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 🔀 Ограничим тренировочный датасет до 10 000 примеров
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(500))

# 🤖 Создаём объект TrainingArguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",        # 📁 Куда сохранять модель
    logging_dir="./logs",                 # 📁 Логи
    logging_steps=10,                     # 📝 Лог каждые 10 шагов
    per_device_train_batch_size=4,        # ⚙️ Batch size побольше (если позволяет память)
    num_train_epochs=1,                   # 🔁 Только 1 эпоха (хватит для демонстрации)
    save_strategy="epoch",                # 💾 Сохраняем модель после каждой эпохи
    report_to="wandb",                    # 📊 Логируем в wandb
    push_to_hub=True,                     # ☁️ Загружаем на Hugging Face
    hub_model_id="tet-ana/gpt2-finetuned" # 🆔 Имя модели на Hugging Face
)

# 🧩 Коллектор данных (без маскирования, для авто-регрессии)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 🏋️‍♀️ Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 🚀 Запускаем обучение
trainer.train()


<ipython-input-21-7f8d1eeb7c04>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,4.906000
20,4.947300
30,5.062500
40,5.120800
50,5.238800
60,4.954300
70,5.019100
80,5.105400
90,4.725800
100,5.156700


TrainOutput(global_step=125, training_loss=4.998496734619141, metrics={'train_runtime': 1374.5602, 'train_samples_per_second': 0.364, 'train_steps_per_second': 0.091, 'total_flos': 32661504000000.0, 'train_loss': 4.998496734619141, 'epoch': 1.0})

In [22]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/tet-ana/gpt2-finetuned/commit/9fd7f8098e5f0c8c7bb2e92de94d84c48d83ef5c', commit_message='End of training', commit_description='', oid='9fd7f8098e5f0c8c7bb2e92de94d84c48d83ef5c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tet-ana/gpt2-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='tet-ana/gpt2-finetuned'), pr_revision=None, pr_num=None)

In [23]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "tet-ana/gpt2-finetuned"  # замени на своё имя на Hugging Face
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()


tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [24]:
import torch

def generate_text(prompt, max_length=100, temperature=1.0, top_k=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



In [25]:
prompt = "Once upon a midnight dreary,"
generated = generate_text(prompt)
print(generated)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a midnight dreary, dark, and lonely stream; yet, at the gale, and his sight--I shall not say. I see nothing; I see no fire nor flames. My dreamless flight lies, as a dreamless star. My dreamless flight lies, as an urn;--and my dreamless life--I come again, with full--full freedom--And we shall end with a new life----and new hope--and new, full--
